# Arena Sport

## Exploiting the Data Leakage in JS dehydratedState

In [3]:
import json
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

def find_products_in_html():
    url = "https://arenakit.net/shop"
    page = urlopen(url)
    html_bytes = page.read()
    html = html_bytes.decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    required_element = None
    for i in soup.descendants:
        if i.getText().count("{\"state\":{\"data\":{\"products\":") != 0:
            required_element = i
    raw_text = required_element.getText()

    start_index = raw_text.find('{"state":{"data":{"products":')
    if start_index == -1:
        raise ValueError("لم يتم العثور على بداية JSON المطلوبة")

    json_part = raw_text[start_index:]

    brace_count = 0
    end_index = None
    for i, ch in enumerate(json_part):
        if ch == '{':
            brace_count += 1
        elif ch == '}':
            brace_count -= 1
            if brace_count == 0:
                end_index = i + 1
                break

    if end_index is None:
        raise ValueError("لم يتم العثور على نهاية JSON")

    clean_json = json_part[:end_index]

    data = json.loads(clean_json)

    products = data["state"]["data"]["products"]
    return products

def extract_variants_per_row(products):
    rows = []
    for p in products:
        name = p.get("name", "")
        size_names = []
        for opt in p.get("productOptions", []):
            if 'size' in (opt.get("name") or "").lower() or 'size' in (opt.get("option", {}).get("name") or "").lower():
                size_names = opt.get("values", []) or []
                break
        variants = p.get("variants", [])
        for i, var in enumerate(variants):
            size = size_names[i] if i < len(size_names) else var.get("sku") or f"Variant {i+1}"
            available = (var.get("quantity", 0) > 0)
            waiting = len(var.get("notifyInStockList", []) or [])
            price_cents = var.get("priceCents")
            if price_cents is None:
                price_cents = p.get("priceCents", 0)
            price = price_cents / 100 if price_cents is not None else None
            disc_cents = var.get("discountedPriceCents")
            if disc_cents is None:
                disc_cents = p.get("discountedPriceCents", 0)
            discounted_price = (disc_cents or 0) / 100 if disc_cents is not None else None
            has_discount = False
            if discounted_price and price is not None:
                has_discount = discounted_price < price
            on_sale = var.get("isOnSale", p.get("isOnSale", False))
            rows.append({
                "name": name,
                "size": size,
                "available": available,
                "waiting": waiting,
                "price": price,
                "on_sale": on_sale,
                "discounted_price": discounted_price if has_discount else None,
            })
    return pd.DataFrame(rows)

df = extract_variants_per_row(find_products_in_html())

[{'id': 1002581, 'barCode': '', 'companyId': 20612, 'defaultVariant': None, 'description': '', 'discountedPriceCents': 0, 'googleProductCategory': None, 'handle': '4803a5c2106b', 'hasExtras': False, 'hideReviews': False, 'isLimited': False, 'isOnSale': False, 'isTracked': True, 'maxQuantity': 1000, 'minQuantity': 1, 'name': 'Tracksuit cafe  - Real Madried - 26', 'priceCents': 49900, 'pricedBy': 'unit', 'pricingUnit': '', 'productOptions': [{'id': 1716470, 'name': 'Sizes ', 'values': ['M', 'L', 'XL', 'XXL'], 'option': {'id': 1, 'name': 'size', 'deleted': False, 'createdAt': '2019-11-21T15:52:48.747+02:00', 'updatedAt': '2019-11-21T15:52:48.747+02:00'}}], 'quantity': 27, 'ribbonLabel': '', 'secondaryThumbUrls': ['https://bucket.zammit.shop/active-storage/icj0p956pktacmikr0kriokyeb0s', 'https://bucket.zammit.shop/active-storage/y0z9u1bjkx4b70cqtm67qacxfjse', 'https://bucket.zammit.shop/active-storage/t7f7hbm1jqruuivp1rnm8cduro3t', 'https://bucket.zammit.shop/active-storage/4t1gpmergzq2ycj

In [18]:
df.to_csv("scrapped_websites/arenakit_data.csv", index=False, encoding="utf-8-sig")
df.to_excel("scrapped_websites/arenakit_data.xlsx", index=False)

## Normal Scrapping of Arenakit for HTML

In [28]:
product_cards = soup.find_all('div', class_='mantine-ltr-Card-root')
len(product_cards)

12

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

products_data = []

def clean_price(price_text):
    if not price_text:
        return None
    cleaned = re.sub(r'[^\d\.\-\s]', '', price_text).strip()
    if ' - ' in cleaned:
        parts = cleaned.split(' - ')
        return max(float(re.sub(r'[^\d\.]', '', p)) for p in parts)
    try:
        return float(cleaned)
    except ValueError:
        return None

options = Options()

driver = webdriver.Chrome(options=options)

CARD_SELECTOR = lambda c: c and 'Paper-root' in c and 'Card-root' in c

for i in range(10):
    url = f"https://arenakit.net/en/shop?page={i}"
    print(f"--- Fetching page {i} using Selenium... ---")

    try:
        driver.get(url)

        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "mantine-ltr-bdn569"))
        )

        time.sleep(2)

        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")

        product_cards = soup.find_all('div', class_=CARD_SELECTOR)

    except Exception as e:
        print(f"Error processing page {i}: {e}")
        continue

    if not product_cards:
        print(f"Warning: No product cards found on page {i}. Assuming end of list.")
        break

    for card in product_cards:
        product_entry = {
            'اسم المنتج': None,
            'السعر الأصلي (EGP)': None,
            'هل يوجد خصم؟': False,
            'السعر بعد الخصم (EGP)': None,
            'متاح حالياً؟': True
        }

        title_tag = card.find('div', class_='mantine-ltr-bdn569')
        if title_tag:
            product_entry['اسم المنتج'] = title_tag.text.strip()

        status_tag = card.find('span', class_='mantine-ltr-Badge-inner')
        if status_tag:
            status_text = status_tag.text.strip()
            if status_text == 'Sold out':
                product_entry['متاح حالياً؟'] = False
            elif status_text == 'Sale':
                product_entry['هل يوجد خصم؟'] = True

        price_container = card.find('div', class_='mantine-ltr-17h4nqu')

        if price_container:
            price_tags = price_container.find_all('div', class_='mantine-ltr-Text-root')

            if len(price_tags) > 1:
                original_price_text = price_tags[0].text.strip()
                sale_price_text = price_tags[1].text.strip()

                product_entry['السعر الأصلي (EGP)'] = clean_price(original_price_text)
                product_entry['السعر بعد الخصم (EGP)'] = clean_price(sale_price_text)
                product_entry['هل يوجد خصم؟'] = True

            elif len(price_tags) == 1:
                single_price = clean_price(price_tags[0].text.strip())
                product_entry['السعر الأصلي (EGP)'] = single_price

        products_data.append(product_entry)

driver.quit()

df = pd.DataFrame(products_data)

KeyboardInterrupt: 

In [46]:
df

,اسم المنتج,السعر الأصلي (EGP),هل يوجد خصم؟,السعر بعد الخصم (EGP),متاح حالياً؟
0,Pyramids - White - 26,325.0,False,NaN,False
1,Roma - White - 26,325.0,False,NaN,False
2,Chelsea - Black - 26,325.0,False,NaN,False
3,Argentina T-shirt (Batistuta) - 1998,499.0,False,NaN,False
4,England Home Shirt - 1998,499.0,False,NaN,True
...,...,...,...,...,...
70,Original Adidas Real Madrid sports jacket - gr...,700.0,True,650.0,False
71,Original Adidas Germany sports jacket - black ...,700.0,False,NaN,False
72,Egyptian Al-Ahly jacket 2024/2025,500.0,True,450.0,False
73,"England 1998 World Cup Home Jersey, Beckham",450.0,False,NaN,False


In [47]:
df.to_csv("scrapped_websites/arenakit_data_normal.csv", index=False, encoding="utf-8-sig")
df.to_excel("scrapped_websites/arenakit_data_normal.xlsx", index=False)


# Scrapping Curva Website

## Scrapping All T-shirts

In [ ]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def scrape_curvaegypt_to_dataframe():
    URL = "https://curvaegypt.com/categories?categoryId=1"

    # المفتاح الثابت الذي أشرت إليه
    NUXT_DATA_KEY = "9H-LPkPSpIELVE57tr3W5KsKD-WmLHzZWYn_4mMr03Y"

    driver = webdriver.Chrome()

    all_products_data = []

    print("بدء عملية السكرابنج للموقع وتحميل الصفحة الأولى...")
    driver.get(URL)
    last_page = 45
    print("فشل العثور على عنصر التنقل. سيتم استخدام الحد الأقصى 45 صفحة.")

    MAX_PAGES_TO_SCRAPE = min(last_page, 45)

    for page_num in range(1, MAX_PAGES_TO_SCRAPE + 1):
        print(f"جار استخراج البيانات من الصفحة رقم {page_num}...")

        # استخدام المفتاح الثابت
        js_script = f"return window.__NUXT__.data['{NUXT_DATA_KEY}'].data.data;"

        try:
            products_list = driver.execute_script(js_script)
        except:
            print(f"فشل استخراج بيانات الكائن في الصفحة {page_num}. محاولة تجاوز هذه الصفحة.")
            products_list = []

        if products_list:
            for product in products_list:

                is_available = product.get("availability") == "available"
                has_discount = product.get("offer_ratio") is not None

                product_data = {
                    "name": product.get("name"),
                    "original price": product.get("init_price"),
                    "discount": has_discount,
                    "discounted price": product.get("offer_price"),
                    "available": is_available
                }
                all_products_data.append(product_data)

        if page_num < MAX_PAGES_TO_SCRAPE:
          #  try:
                # 1. إيجاد زر التنقل
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//button[@aria-label='Go to next page']"))
                )

                # 2. النقر بواسطة JavaScript
                driver.execute_script("arguments[0].click();", next_button)

                # 3. الانتظار الصريح لمدة 3 ثواني (بدلاً من الانتظار المعقد)
                time.sleep(3)

         #   except:
          #      print("تم الوصول للصفحة الأخيرة أو فشل العثور على زر التنقل.")
           #     break

    driver.quit()

    df = pd.DataFrame(all_products_data)
    return df

products_df = scrape_curvaegypt_to_dataframe()

print("\n--- الداتا فريم الناتجة (أول 10 صفوف) ---")
print(products_df.head(10).to_string())

print(f"\nتم استخراج {len(products_df)} منتجاً بنجاح.")

بدء عملية السكرابنج للموقع وتحميل الصفحة الأولى...
فشل العثور على عنصر التنقل. سيتم استخدام الحد الأقصى 45 صفحة.
جار استخراج البيانات من الصفحة رقم 1...
جار استخراج البيانات من الصفحة رقم 2...
جار استخراج البيانات من الصفحة رقم 3...
جار استخراج البيانات من الصفحة رقم 4...
جار استخراج البيانات من الصفحة رقم 5...
جار استخراج البيانات من الصفحة رقم 6...
جار استخراج البيانات من الصفحة رقم 7...
جار استخراج البيانات من الصفحة رقم 8...
جار استخراج البيانات من الصفحة رقم 9...
جار استخراج البيانات من الصفحة رقم 10...
جار استخراج البيانات من الصفحة رقم 11...
جار استخراج البيانات من الصفحة رقم 12...
جار استخراج البيانات من الصفحة رقم 13...
جار استخراج البيانات من الصفحة رقم 14...
جار استخراج البيانات من الصفحة رقم 15...
جار استخراج البيانات من الصفحة رقم 16...
جار استخراج البيانات من الصفحة رقم 17...
جار استخراج البيانات من الصفحة رقم 18...
جار استخراج البيانات من الصفحة رقم 19...
جار استخراج البيانات من الصفحة رقم 20...
جار استخراج البيانات من الصفحة رقم 21...
جار استخراج البيانات من الصفحة رقم 

In [3]:
products_df.to_csv("scrapped_websites/curva_basic.csv", index=False, encoding="utf-8-sig")
products_df.to_excel("scrapped_websites/curva_basic.xlsx", index=False)


In [4]:
t_shirts_df = products_df[products_df['name'].str.contains('قميص', na=False)]

In [5]:
t_shirts_df

,name,original price,discount,discounted price,available
0,قميص الزمالك الاحتياطي 2025/26 (عرض خاص),125,False,NaN,True
1,قميص مانشستر سيتي الاحتياطي 2025/26 بشعارات تط...,300,True,195.0,True
2,قميص ليفربول الثالث 2025/26 مع بادجات الدوري ا...,300,False,NaN,True
3,قميص مانشستر يونايتد الاحتياطي 2025/26 بشعارات...,300,False,NaN,True
4,قميص وادي دجلة الأساسي 2025/26 بشعارات تطريز B...,300,False,NaN,True
...,...,...,...,...,...
1309,قميص الأهلي التخيلي 2022 - النسخة الثالثة,205,False,NaN,False
1310,قميص الأهلي التخيلي 2022 - النسخة الأولى,205,False,NaN,False
1311,قميص الزمالك التخيلي 2022 - النسخة الخامسة,205,False,NaN,False
1340,قميص الزمالك الأساسي 2020/21 بالرعايا,145,False,NaN,True


In [6]:
t_shirts_df.to_excel("scrapped_websites/curva_basic.xlsx", index=False)

## Scrapping Bestseller T-shirts

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def scrape_curvaegypt_to_dataframe():
    URL = "https://curvaegypt.com/about/top-products"

    # المفتاح الثابت الذي أشرت إليه
    NUXT_DATA_KEY = "9H-LPkPSpIELVE57tr3W5KsKD-WmLHzZWYn_4mMr03Y"

    driver = webdriver.Chrome()

    all_products_data = []

    print("بدء عملية السكرابنج للموقع وتحميل الصفحة الأولى...")
    driver.get(URL)

    MAX_PAGES_TO_SCRAPE = 127

    for page_num in range(1, MAX_PAGES_TO_SCRAPE + 1):
        print(f"جار استخراج البيانات من الصفحة رقم {page_num}...")

        # استخدام المفتاح الثابت
        js_script = f"return window.__NUXT__.data['{NUXT_DATA_KEY}'].data.data;"

        try:
            products_list = driver.execute_script(js_script)
        except:
            print(f"فشل استخراج بيانات الكائن في الصفحة {page_num}. محاولة تجاوز هذه الصفحة.")
            products_list = []

        if products_list:
            for product in products_list:

                is_available = product.get("availability") == "available"
                has_discount = product.get("offer_ratio") is not None
                if 'قميص' not in product.get("name"):
                    continue

                product_data = {
                    "name": product.get("name"),
                    "original price": product.get("init_price"),
                    "discount": has_discount,
                    "discounted price": product.get("offer_price"),
                    "available": is_available
                }
                all_products_data.append(product_data)

        if page_num < MAX_PAGES_TO_SCRAPE:
          #  try:
                # 1. إيجاد زر التنقل
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//button[@aria-label='Go to next page']"))
                )

                # 2. النقر بواسطة JavaScript
                driver.execute_script("arguments[0].click();", next_button)

                # 3. الانتظار الصريح لمدة 3 ثواني (بدلاً من الانتظار المعقد)
                time.sleep(3)

         #   except:
          #      print("تم الوصول للصفحة الأخيرة أو فشل العثور على زر التنقل.")
           #     break

    driver.quit()

    df = pd.DataFrame(all_products_data)
    return df

products_df_sorted_by_importance = scrape_curvaegypt_to_dataframe()

print("\n--- الداتا فريم الناتجة (أول 10 صفوف) ---")
print(products_df_sorted_by_importance.head(10).to_string())

print(f"\nتم استخراج {len(products_df_sorted_by_importance)} منتجاً بنجاح.")

بدء عملية السكرابنج للموقع وتحميل الصفحة الأولى...
جار استخراج البيانات من الصفحة رقم 1...
جار استخراج البيانات من الصفحة رقم 2...
جار استخراج البيانات من الصفحة رقم 3...
جار استخراج البيانات من الصفحة رقم 4...
جار استخراج البيانات من الصفحة رقم 5...
جار استخراج البيانات من الصفحة رقم 6...
جار استخراج البيانات من الصفحة رقم 7...
جار استخراج البيانات من الصفحة رقم 8...
جار استخراج البيانات من الصفحة رقم 9...
جار استخراج البيانات من الصفحة رقم 10...
جار استخراج البيانات من الصفحة رقم 11...
جار استخراج البيانات من الصفحة رقم 12...
جار استخراج البيانات من الصفحة رقم 13...
جار استخراج البيانات من الصفحة رقم 14...
جار استخراج البيانات من الصفحة رقم 15...
جار استخراج البيانات من الصفحة رقم 16...
جار استخراج البيانات من الصفحة رقم 17...
جار استخراج البيانات من الصفحة رقم 18...
جار استخراج البيانات من الصفحة رقم 19...
جار استخراج البيانات من الصفحة رقم 20...
جار استخراج البيانات من الصفحة رقم 21...
جار استخراج البيانات من الصفحة رقم 22...
جار استخراج البيانات من الصفحة رقم 23...
جار استخراج الب

In [11]:
products_df_sorted_by_importance.to_excel("scrapped_websites/curva_bestseller.xlsx", index=False)

# Africa Store

In [2]:
url = "https://africastoreas.com/products?category_id=8&data_from=category&page=5"
page = urlopen(url)
html_bytes = page.read()
html = html_bytes.decode('utf-8')
html

'<!DOCTYPE html>\n<html lang="ar" dir="rtl">\n<head>\n\n    <meta name="base-url" content="https://africastoreas.com">\n    <meta property="og:site_name" content="Africa Store" />\n\n    <meta name="google-site-verification" content="&lt;meta name=&quot;google-site-verification&quot; content=&quot;TdUTNWvPs10mD3aYMt4c7ubFDEv-w63VH-eHZlxyn5M&quot; /&gt;">\n    <meta name="msvalidate.01" content="">\n    <meta name="baidu-site-verification" content="">\n    <meta name="yandex-verification" content="">\n\n    <meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8"/>\n    <meta name="viewport" content="width=device-width, initial-scale=1"/>\n    <meta name="robots" content="index, follow">\n    <meta name="_token" content="dJHYV111aHMvvFTYPLsyqDC2LngQCWe2lTiMqLEb">\n    <link rel="shortcut icon" href="https://africastoreas.com/storage/app/public/company/2025-07-24-68817f1d1b841.webp"/>\n\n    <link rel="stylesheet" href

In [4]:
soup = BeautifulSoup(html, 'html.parser')
soup.prettify()

'<!DOCTYPE html>\n<html dir="rtl" lang="ar">\n <head>\n  <meta content="https://africastoreas.com" name="base-url"/>\n  <meta content="Africa Store" property="og:site_name">\n   <meta content=\'&lt;meta name="google-site-verification" content="TdUTNWvPs10mD3aYMt4c7ubFDEv-w63VH-eHZlxyn5M" /&gt;\' name="google-site-verification"/>\n   <meta content="" name="msvalidate.01"/>\n   <meta content="" name="baidu-site-verification"/>\n   <meta content="" name="yandex-verification"/>\n   <meta content="IE=edge" http-equiv="X-UA-Compatible">\n    <meta content="text/html; charset=utf-8" http-equiv="content-type">\n     <meta content="width=device-width, initial-scale=1" name="viewport">\n      <meta content="index, follow" name="robots"/>\n      <meta content="dJHYV111aHMvvFTYPLsyqDC2LngQCWe2lTiMqLEb" name="_token"/>\n      <link href="https://africastoreas.com/storage/app/public/company/2025-07-24-68817f1d1b841.webp" rel="shortcut icon"/>\n      <link href="https://africastoreas.com/resources/th

In [7]:
base_url = "https://africastoreas.com/products?category_id=8&data_from=category&page="
products_data = []
max_pages = 9

for page_num in range(1, max_pages + 1):
    url = f"{base_url}{page_num}"
    print(f"Scraping page: {page_num}")

    try:
        response = requests.get(url, verify=False)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        product_containers = soup.find_all('div', class_='product')

        if not product_containers:
            print(f"No products found on page {page_num}. Stopping loop.")
            break

        for container in product_containers:
            product_name = "Name not available"
            product_price = "Price not available"

            try:
                title_tag = container.find('h6', class_='product__title')
                product_name_tag = title_tag.find('a')
                product_name = product_name_tag.text.strip()
            except AttributeError:
                pass

            try:
                price_tag = container.find('ins', class_='product__new-price')
                product_price = price_tag.text.strip()
            except AttributeError:
                pass

            products_data.append({
                "name": product_name,
                "original price": product_price,
                "discount": None,
                "discounted price": None,
                "available": None
            })


    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {page_num}: {e}")
        break

print("\n--- Scraping is Done ---")

Scraping page: 1


C:\Users\20106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'africastoreas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Scraping page: 2


C:\Users\20106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'africastoreas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Scraping page: 3


...

Scraping page: 8


C:\Users\20106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'africastoreas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Scraping page: 9


C:\Users\20106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'africastoreas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



--- Scraping is Done ---


In [9]:
import pandas as pd

df = pd.DataFrame(products_data)
df.to_excel("scrapped_websites/africa_store.xlsx", index=False)